In [9]:
# coding: utf-8
# In[110]:

from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances


import pandas as pd
get_ipython().magic('matplotlib')


# In[111]:

df_all=pd.read_csv('C:/diabetic_data_processed.csv')
print(type(df_all))


#df_all=pd.read_csv('C:/gcat_epi_genomic.csv')
#df=pd.read_csv('C:/binary_2016_11_14.csv')

#df_all = df_all[df_all['ENFERMEDADES_DIABETES_T2DM']==1]
msk = np.random.rand(len(df_all)) < 0.8
df, df_test = df_all[msk].copy(deep = True), df_all[~msk].copy(deep = True)
df = df.reset_index()
df_test = df_test.reset_index()
print("df.shape",df.shape)
print("df_test.shape",df_test.shape)
"""
msk = np.random.rand(len(df_all)) < 0.8

df= df_all[msk]
print("Longitud",len(df))

df_test = df_all[~msk]
print("Longitud2",len(df_test))
"""

y_train=df['readmitted']
y_test=df_test['readmitted']

print("Test", set(y_test))

# In[112]:

to_del = ['encounter_id', 'patient_nbr','medical_specialty','payer_code','weight','index']

#to_del = ['entity_id','sampleType','FECHA_NACIMIENTO','MUNICIPIO_RESIDENCIA',
#          'MUNICIPIO_NACIMIENTO','PROVINCIA_RESIDENCIA','PAIS_NACIMIENTO', 'CALC_AVG_HEIGHT','CALC_AVG_PESO','CALC_AVG_CINTURA','CALC_AVG_CADERA', 
#        'CRIBADO_PSA_RESULTADO_ANTERIOR_2','CRIBADO_PSA_RESULTADO_ANTERIOR_0' ,'CRIBADO_PSA_RESULTADO_1',
#         'CRIBADO_PSA_RESULTADO_0','CRIBADO_PSA_2','CRIBADO_PSA_1','CRIBADO_1','CRIBADO_2','CRIBADO_HECES']

#to_del = ['CRIBADO_HECES_2','Entity_id','0_IID','entity_id','sampleType','FECHA_NACIMIENTO','MUNICIPIO_RESIDENCIA',
#          'MUNICIPIO_NACIMIENTO','PROVINCIA_RESIDENCIA','PAIS_NACIMIENTO', 'CALC_AVG_HEIGHT','CALC_AVG_PESO','CALC_AVG_CINTURA','CALC_AVG_CADERA', 
#        'CRIBADO_PSA_RESULTADO_ANTERIOR_2','CRIBADO_PSA_RESULTADO_ANTERIOR_0' ,'CRIBADO_PSA_RESULTADO_1',
#         'CRIBADO_PSA_RESULTADO_0','CRIBADO_PSA_2','CRIBADO_PSA_1','CRIBADO_1','CRIBADO_2','ETNIA_MADRE_6',
#         'ETNIA_PARTICIPANTE_1','ETNIA_PARTICIPANTE_6','ETNIA_PADRE_1','ETNIA_PADRE_6','ETNIA_MADRE_1','index']
           
"""
to_del.extend(['ENFERMEDADES_DIABETES','ENFERMEDADES_HTA',
              'ENFERMEDADES_HIPERCOLESTEROLEMIA','ENFERMEDADES_ICTUS',
              'ENFERMEDADES_INFARTO','ENFERMEDADES_ANGINA'
              ])
"""

print (to_del)

#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
print ("df_2",df_2.shape)

# Filter complete null columns
cols = np.where((np.sum(df_2.isnull(), axis=0).values) == df_2.shape[0])[0]
print (cols)
filt_cols = [c for c in df_2.columns if c not in df_2.columns[cols]]
df_3 = df_2[filt_cols]
print ("df_3",df_3.shape)

#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)

scaler = preprocessing.MinMaxScaler().fit(df_4[:,0:-2])
data = scaler.transform(df_4[:,0:-2])
data2 = df_4
print ("data",data.shape)

#Test
#Filter_selected cols
filtered_cols = [c for c in df_test.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2_test = df_test[filtered_cols]
print ("df_2",df_2_test.shape)

# Filter complete null columns
cols = np.where((np.sum(df_2_test.isnull(), axis=0).values) == df_2_test.shape[0])[0]
print (cols)
filt_cols = [c for c in df_2_test.columns if c not in df_2_test.columns[cols]]
df_3_test = df_2_test[filt_cols]
print ("df_3",df_3_test.shape)

#Fill na
df_4_test = df_3_test.fillna(value=np.mean(df_3_test,axis=0),inplace=False,axis=0).values
print ("df_4",df_4_test.shape)

scaler_test = preprocessing.MinMaxScaler().fit(df_4_test[:,0:-2])
data_test = scaler_test.transform(df_4_test[:,0:-2])
print ("data_test",data_test.shape)

from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=(.999 * (1 - .999)))
newdata=selector.fit_transform(data)
idxs = selector.get_support(indices=True)
#print(data[:, idxs])
#print("indices",idxs)
columnslist=df_3.columns.tolist()
print("lenindex",len(idxs))
#for z in range(0,len(columnslist)):
    #if z not in idxs:
        #print(columnslist[z])
print("after",newdata.shape)
print("initial",data.shape)
newdata_test=np.zeros((len(df_test), len(newdata[0])))
for y in range (0,len(df_test)):
    count=0
    for z in range(0,len(columnslist)):
        if z in idxs:
            #print("y",y,"z",z)
            newdata_test[y][count]=data_test[y][z]
            count=count+1
print("test_final",newdata_test.shape)

Using matplotlib backend: Qt5Agg


C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
df.shape (81528, 51)
df_test.shape (20238, 51)
Test {0, 29, 31}
['encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code', 'weight', 'index']
df_2 (81528, 45)
[]
df_3 (81528, 45)
df_4 (81528, 45)
data (81528, 43)
df_2 (20238, 45)
[]
df_3 (20238, 45)
df_4 (20238, 45)
data_test (20238, 43)
lenindex 27
after (81528, 27)
initial (81528, 43)
test_final (20238, 27)


In [12]:
x_train=data[:,:-2]
x_test=data_test[:,:-2]
print(set(y_train))
print(set(y_test))
print(df_3.columns)

{0, 29, 31}
{0, 29, 31}
Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [13]:
#Random Forest
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

x_train=data[:,:-2]
y_train=data[:,-1]
x_test=data_test[:,:-2]
y_test=data_test[:,-1]

rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=2, oob_score = True) 

# Set the parameters by cross-validation
#'min_samples_leaf': [1,5,10,50,100,200,500],

tuned_parameters = {
    'n_estimators': [20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'class_weight':['balanced']
}

#tuned_parameters = {  
#    }  

#scores = ['precision', 'recall', 'f1']
#scores = [ 'f1_weighted','f1_micro','f1_macro','accuracy','roc_auc']
scores=['f1_weighted']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    #clf = GridSearchCV(GaussianNB(), tuned_parameters, cv=10,scoring='%s_macro' % score)
    #clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s_weighted' % score,n_jobs= -1)
    clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s' % score,n_jobs= -1)
    clf.fit(x_train, y_train)
    array=clf.best_estimator_.feature_importances_
    print(type(array))
    for i in range (0,len(array)):
        print(df_2.columns[i],": ",array[i])
    #print("clf: ",type(clf))
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_train, y_pred = y_test, clf.predict(x_test)
    y_train2, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_train2, y_pred))
    print("Accuracy score: ", accuracy_score(y_test, y_pred))
    print()

# Tuning hyper-parameters for f1_weighted

<class 'numpy.ndarray'>
race :  0.00361191605591
gender :  0.00266165375904
age :  0.00821530948067
admission_type_id :  0.00533308526263
discharge_disposition_id :  0.00597508543637
admission_source_id :  0.00444444625956
time_in_hospital :  0.0107430658081
num_lab_procedures :  0.0147921312914
num_procedures :  0.00645620101754
num_medications :  0.0345220659746
number_outpatient :  0.00383931646627
number_emergency :  0.00304541973463
number_inpatient :  0.00488584830672
diag_1 :  0.0149433475641
diag_2 :  0.0135230563745
diag_3 :  0.0130652194136
number_diagnoses :  0.0072684905254
max_glu_serum :  0.00177316104317
A1Cresult :  0.00716939819882
metformin :  0.113017350574
repaglinide :  0.00831180166525
nateglinide :  0.00382717990328
chlorpropamide :  0.000478782574287
glimepiride :  0.0293509911273
acetohexamide :  2.51203036568e-05
glipizide :  0.0607273936295
glyburide :  0.0603470630946
tolbutamide :  0.000124673637254
pioglitazone : 